In [1]:
import tensorflow as tf

import numpy as np
import os
import time

Download the data

In [2]:
#  Used the following data at the given link.
filename = 'tramp10.txt'
link_to_file = 'http://textfiles.com/etext/AUTHORS/TWAIN/tramp10.txt' 

path_to_file = tf.keras.utils.get_file(filename, link_to_file)

Read the data

First, look in the text:

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 887067 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

Please take a look at the important information in this header.
We encourage you to keep this file on your own disk, keeping an
electronic path open for the next readers.  Do not remove this.


**Welcome To The World of Free Plain Vanilla Electr


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

90 unique characters


In [6]:
print(vocab)

['\n', '\r', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~']


Process the text

Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [8]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

P
l
e
a
s


The batch method lets us easily convert these individual characters to sequences of the desired size.

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Please take a look at the important information in this header.\r\nWe encourage you to keep this file o'
'n your own disk, keeping an\r\nelectronic path open for the next readers.  Do not remove this.\r\n\r\n\r\n**W'
'elcome To The World of Free Plain Vanilla Electronic Texts**\r\n\r\n**Etexts Readable By Both Humans and '
'By Computers, Since 1971**\r\n\r\n*These Etexts Prepared By Hundreds of Volunteers and Donations*\r\n\r\nInfo'
'rmation on contacting Project Gutenberg to get Etexts, and\r\nfurther information is included below.  W'


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [10]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [11]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Please take a look at the important information in this header.\r\nWe encourage you to keep this file '
Target data: 'lease take a look at the important information in this header.\r\nWe encourage you to keep this file o'


Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [12]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

Build The Model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;

tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)

tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [13]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 512     # Change-1 : Increased the embedding dimension so that the output dimension of the first layer will be large enough for better computation speed and results 

# Number of RNN units
rnn_units = 2048        # Change-2 : Increased the units in the GRU layer so that there will be more neurons which will help increase the accuracy.

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.Dense(vocab_size),                                 # Change-3: Added a new Dense layer so that accuracy will be increased by adding a fully connected layer.
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.2),                                     # Change-4 : Added a dropout layer so that it will help reducing the overfitting of the model
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [16]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [17]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 90) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 512)           46080     
_________________________________________________________________
dense (Dense)                (64, None, 90)            46170     
_________________________________________________________________
gru (GRU)                    (64, None, 2048)          13148160  
_________________________________________________________________
dropout (Dropout)            (64, None, 2048)          0         
_________________________________________________________________
dense_1 (Dense)              (64, None, 90)            184410    
Total params: 13,424,820
Trainable params: 13,424,820
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [19]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
#This gives us, at each timestep, a prediction of the next character index:
sampled_indices

array([15, 18, 32, 32, 44, 42, 30, 65, 11, 66, 51, 18, 79, 67, 35, 63, 10,
       20, 23,  7, 13, 12, 84, 49, 60, 10, 13,  8, 60, 43, 23, 37, 16, 41,
       69, 71, 75, 61, 32, 18, 23, 51, 51, 63, 85, 51, 80, 51, 78, 32, 62,
       85, 11, 65, 38, 65, 40, 78,  1, 32, 15, 64, 73, 66, 43, 25, 63, 12,
       23, 45,  8, 77, 82, 66, 53, 78, 18, 58, 74,  7, 64, 14, 12,  9, 51,
       70, 57, 22, 40, 33, 36,  2, 15, 13, 41, 63, 22, 45, 27, 41])

Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

Attach an optimizer, and a loss function
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [20]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 90)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4992294


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [21]:
model.compile(optimizer='adam', loss=loss)

Configure checkpoints

Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [22]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Execute the training

To keep training time reasonable, use 10 epochs to train the model. 

In [23]:
EPOCHS=22  # Change-5: Determined the best number of epochs emprically so that it gives the mininum loss value. This will definitely affect the results in a positive way.
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/22
137/137 [==============================] - 13s 98ms/step - loss: 2.8194
Epoch 2/22
137/137 [==============================] - 14s 99ms/step - loss: 1.9506
Epoch 3/22
137/137 [==============================] - 14s 99ms/step - loss: 1.6849
Epoch 4/22
137/137 [==============================] - 13s 98ms/step - loss: 1.5306
Epoch 5/22
137/137 [==============================] - 13s 98ms/step - loss: 1.4350
Epoch 6/22
137/137 [==============================] - 13s 98ms/step - loss: 1.3624
Epoch 7/22
137/137 [==============================] - 13s 98ms/step - loss: 1.2975
Epoch 8/22
137/137 [==============================] - 13s 98ms/step - loss: 1.2381
Epoch 9/22
137/137 [==============================] - 13s 98ms/step - loss: 1.1814
Epoch 10/22
137/137 [==============================] - 13s 97ms/step - loss: 1.1207
Epoch 11/22
137/137 [==============================] - 13s 97ms/step - loss: 1.0584
Epoch 12/22
137/137 [==============================] - 13s 97ms/step - loss: 0.9943
E

Generate text

Restore the latest checkpoint
To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [24]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 512)            46080     
_________________________________________________________________
dense_2 (Dense)              (1, None, 90)             46170     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 2048)           13148160  
_________________________________________________________________
dropout_1 (Dropout)          (1, None, 2048)           0         
_________________________________________________________________
dense_3 (Dense)              (1, None, 90)             184410    
Total params: 13,424,820
Trainable params: 13,424,820
Non-trainable params: 0
_________________________________________________________________


The prediction loop

The following code block generates the text:

It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

Get the prediction distribution of the next character using the start string and the RNN state.

Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [25]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [26]:
print(generate_text(model, start_string=u"Frankfort "))

Frankfort a patheth with it; and all the as werr
sending an explanatory some of the house. 
Mr. Garmans for re,uce of the lefthess go without
the Harris happy and the Picrd step, actor, and every time. 

The young fellow winds,
but all that aided me was about to wreathe manick and suffering
recorded the second out of the five corps.  A year layter,
and within two minutes afterward with their beguiling
and impressive charmed a mournest forth
or h muttered, rup the bitter
with a college jewsp; and what sort of time they really survived, and the great
German tenm of your finger till seem for suring a young lady with the other sed
to come, and presently passed close to making the
summit and lead him to go up and the above edge
has it of compulsion to stay thirty-t was
the work of the Swere even poor Merich,
and his talk over the precipice, and he had done a bright Council
has any charmed off you were to come," we were
two beds and give single ray, and night, and the door
was sour
